#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### CHAPTER 22
**CH22A How does a merger between airlines affect prices?**

 using the airline-tickets-usa dataset
 version 0.9 2020-09-11

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import sys
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
# Current script folder
current_path = os.getcwd()
dirname = "/".join(current_path.split("/")[:-2]) + "/"

# location folders
data_in = dirname + "da_data_repo/airline-tickets-usa/clean/"
data_out = dirname + "da_case_studies/ch22-airline-merger-prices/"
output = dirname + "da_case_studies/ch22-airline-merger-prices/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)

In [3]:
# Import the prewritten helper functions 
from py_helper_functions import *

## Data Preparation Steps and Descriptive Statistics

In [4]:
data = pd.read_stata(data_in+"originfinal-panel.dta")

In [5]:
data = data.query("year==2011 | year==2016").reset_index(drop=True)

In [6]:
# * create total number of passengers from shares
# * so we can get aggreagate shares

data["ptotalAA"] = data["shareAA"] * data["passengers"]
data["ptotalUS"] = data["shareUS"] * data["passengers"]
data["ptotallargest"] = data["sharelargest"] * data["passengers"]

data_agg = data.groupby(["origin", "finaldest", "return", "year"]).agg(
    airports=("airports", "first"),
    return_sym=("return_sym", "first"),
    stops=("stops", "first"),
    ptotalAA=("ptotalAA", sum),
    ptotalUS=("ptotalUS", sum),
    ptotallargest=("ptotallargest", sum),
    passengers=("passengers", sum),
    itinfare=("itinfare", sum),
).reset_index()

In [7]:
data_agg["after"] = np.where(data_agg["year"] == 2016, 1, 0)
data_agg["before"] = np.where(data_agg["year"] == 2011, 1, 0)
data_agg["avgprice"] = data_agg["itinfare"] / data_agg["passengers"]
data_agg["shareAA"] = data_agg["ptotalAA"] / data_agg["passengers"]
data_agg["shareUS"] = data_agg["ptotalUS"] / data_agg["passengers"]
data_agg["sharelargest"] = data_agg["ptotallargest"] / data_agg["passengers"]
data_agg["AA"] = np.where(data_agg["shareAA"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["AA_and_US"] = np.where(
    (data_agg["shareAA"] > 0) & (data_agg["shareUS"]) > 0, 1, 0
)
data_agg["AA_or_US"] = np.where(
    (data_agg["shareAA"] > 0) | (data_agg["shareUS"]) > 0, 1, 0
)

In [8]:
# create numeric ID for market
data_agg["market"] = (
    data_agg["origin"]
    + "_"
    + data_agg["finaldest"]
    + "_"
    + data_agg["return"].apply(str)
)

In [9]:
# passengers before and after
data_agg["pass_bef_id"] = np.where(data_agg["before"] == 1, data_agg["passengers"], np.nan)
data_agg["pass_aft_id"] = np.where(data_agg["after"] == 1, data_agg["passengers"], np.nan)

data_agg["pass_bef"] = data_agg.groupby("market")["pass_bef_id"].transform(np.nanmean)
data_agg["pass_aft"] = data_agg.groupby("market")["pass_aft_id"].transform(np.nanmean)

In [10]:
# balanced vs unbalanced part of panel
data_agg["balanced"] = data_agg.groupby("market")["origin"].transform("count") == 2

In [11]:
data_agg.groupby("balanced").agg(
    sum_passengers=("passengers", sum), n=("origin", "count")
)

,sum_passengers,n
balanced,,
False,561060.0,51516
True,54136421.0,226094


In [29]:
# Define treated and untreated markets
# treated: both AA and US present in the before period
# untreated: neither AA nor US present in the before period
# drop if only AA or only US in before period (neither treated nor untreated)

In [13]:
def functions_for_groupby(df):
    d = {}
    d["treated"] = np.where(
        df["balanced"] == 1, max((df["AA_and_US"] == 1) & (df["before"] == 1)), None
    )
    d["untreated"] = np.where(
        df["balanced"] == 1, max((df["AA_and_US"] == 0) & (df["before"] == 1)), None
    )
    d["smallmkt"] = max((df["passengers"] < 5000) & (df["before"] == 1))

    return pd.Series(d, index=["treated", "untreated", "smallmkt"])

In [192]:
data_agg.groupby("market").apply(functions_for_groupby)

,treated,untreated,smallmkt
market,,,
ABE_ABE_1,[None],[None],True
ABE_ABI_0,[None],[None],True
ABE_ABI_1,"[False, False]","[True, True]",True
ABE_ABQ_0,"[False, False]","[True, True]",True
ABE_ABQ_1,"[False, False]","[True, True]",True
...,...,...,...
YUM_VPS_1,"[False, False]","[True, True]",True
YUM_XNA_0,[None],[None],False
YUM_XNA_1,"[False, False]","[True, True]",True


In [14]:
data_agg["lnavgp"] = np.where(np.isinf(np.log(data_agg["avgprice"])), np.nan, np.log(data_agg["avgprice"]))
data_agg["d_lnavgp"] = data_agg.groupby("market")["lnavgp"].transform("diff")

## Describe

In [15]:
#describe yearly data
data_agg[["year","passengers"]].groupby("year").describe(percentiles=[.25, .5, .75,.9]).round(0)

passengers                                                    
          count   mean     std  min  25%  50%   75%    90%      max
year                                                               
2011   141712.0  170.0   980.0  1.0  2.0  7.0  37.0  205.0  39853.0
2016   135898.0  225.0  1270.0  1.0  2.0  8.0  45.0  245.0  46881.0

In [16]:
data_agg.query('(origin=="JFK") & (finaldest=="LAX")')[
    ["market", "origin", "finaldest", "return", "year", "passengers"]
]

,market,origin,finaldest,return,year,passengers
139047,JFK_LAX_0,JFK,LAX,0,2011,31312.0
139048,JFK_LAX_0,JFK,LAX,0,2016,46881.0
139049,JFK_LAX_1,JFK,LAX,1,2011,37861.0
139050,JFK_LAX_1,JFK,LAX,1,2016,41927.0


In [17]:
data_agg %>%
  filter(year == 2011) %>%
  select(smallmkt, passengers) %>%
  group_by(smallmkt) %>%
  summarise_each(funs(N = length, 
                      min = min, 
                      max = max,
                      median = median,
                      mean = mean, 
                      sum = sum))

SyntaxError: invalid syntax (<ipython-input-17-79e6c5201c96>, line 1)

In [168]:
# describe balanced
data_agg.groupby(["year", "balanced"])["passengers"].describe(percentiles=[]).round(1)

count   mean     std  min   50%      max
year balanced                                             
2011 False      28665.0    3.6    23.8  1.0   1.0   2051.0
     True      113047.0  211.9  1092.7  1.0  12.0  39853.0
2016 False      22851.0   20.0   178.1  1.0   1.0   4798.0
     True      113047.0  267.0  1386.3  1.0  13.0  46881.0

In [169]:
# describe treatment
data_agg.groupby(["year", "treated", "untreated"]).describe(percentiles=[]).round(1)

KeyError: 'treated'

In [186]:
# describe outcome
data_agg.query("before == 1")["avgprice"].describe().round(1)

count    141712.0
mean        455.7
std         276.5
min           0.0
25%         285.5
50%         413.4
75%         561.0
max        6054.0
Name: avgprice, dtype: float64

In [187]:
data_agg.query("before == 1")["avgprice"].sum().round(1)

64584985.6

In [188]:
data_agg.query("avgprice == 0")["passengers"].describe().round(1)

count    843.0
mean       1.1
std        0.3
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        3.0
Name: passengers, dtype: float64

In [190]:
data_agg.query("avgprice == 0")["passengers"].sum().round(1)

909.0